In [ ]:
# Fig 8 (a): This will give you the Smolyak's strategy surviving prob. for fully protection curve (ER). 
# for comparion, you may use Fig_5 code


import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

###################### ER coloring algorith #########################
def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -1) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 1) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
    
    return None

def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(H.degree())      # in original code, it's "G.degree()"
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict


import networkx as nx

def get_immune_nodes(G, fragile_nodes):
    """
    Generate the connected subgraph of fragile nodes, calculate the average degree 
    of the subgraph, and return the fragile nodes whose degree is greater than the 
    average degree of that subgraph.

    Arguments:
    - G (networkx.Graph): The original graph.
    - fragile_nodes (list): List of fragile nodes in the network.

    Returns:
    - immune_nodes (set): Set of fragile nodes whose degree is greater than the 
                           average degree of the connected subgraph of fragile nodes.
    """
    
    # Create a subgraph of only the fragile nodes
    subgraph = G.subgraph(fragile_nodes).copy()
    
    # Calculate the average degree of the subgraph
    avg_degree = sum(dict(subgraph.degree()).values()) / len(subgraph.nodes())
    
    # Identify immune nodes (nodes with degree greater than the average degree)
    immune_nodes = {node for node in subgraph.nodes() if subgraph.degree(node) > avg_degree}
    
    return immune_nodes





def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    # print(min_value)
    protected_nodes_list = []
    for i in failed_nodes:
        if H.degree(i) >min_value:
            protected_nodes_list.append(i)
    # print(protected_nodes_list)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha, prot_prob):
################################ Cascading failure ######################################
    #print(H.degree())
    #initial_edge = random.choice(list(H.edges))
   
    initial_node = random.choice([i for i in list(H.nodes) if i not in immune_nodes ])
   
    #print("edge removed = ", initial_edge)
    #H.remove_edge(*initial_edge)
    H.remove_node(initial_node)         #randomly removing a node, avoiding the immume_nodes, to initiate the cascade
#     print("node removed =",initial_node)
    #print(H.degree())

    
    not_failed_nodes =[]
    failed_nodes = []
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        #print("current_failed_nodes = ", current_failed_nodes)
        #print("current_failed_nodes degree =", H.degree())
        for each in current_failed_nodes:
            #print("entered for loop: current_failed_nodes")
            if each in immune_nodes:
                if prot_prob ==1.0:
                    pass
                elif prot_prob ==0.0:
                    H.remove_node(each)
                    if each not in failed_nodes:
                        failed_nodes.append(each)
                else:
                    r = np.random.random(10000)
                    event = (r<prot_prob)
                    ratio = np.mean(event)
                    if np.random.choice(event) == False:
                        H.remove_node(each)
                        if each not in failed_nodes:
                            failed_nodes.append(each)
            else :
                #print("entered else remove loop: node removed = ", each)
                #print(each, "Non -immune node removed")
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
#                 current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]

        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}
        #print("not_failed_nodes = ",not_failed_nodes)
        #print("not_failed_nodes_desc = ",not_failed_nodes_desc)
        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or (each in immune_nodes and prot_prob==1.0) or (each in immune_nodes and prot_prob==0.7) or (each in immune_nodes and prot_prob==0.4):
                #print("entered if condition: each>= phi or each in immune")
                count +=1
        if count == len(not_failed_nodes):
            #print("Break: count == len(not_failed_nodes)")
            break
         
    return failed_nodes


N_nodes = 20000
# p = 0.008
m = 80000
average_degree_of_original_graph = (2*m) /N_nodes 
prot_prob = [1.0]

Phi = np.arange(0.85,0.90,0.005)


realizations = 150
immune_node_size =[] 
selected_node_size =[]

for protection_prob in prot_prob:
    suriviving_prob_array = []
    for thetha in Phi:
        intermediate_suriviving_prob_array = []
        intermediate_immune_node_size =[]
        intermediate_selected_node_size =[]
        for i in range(realizations):
            G = nx.gnm_random_graph(N_nodes,m)
            degree_sequence_erdos = list(G.degree())
            degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
            avg_degree = np.mean([int(x) for x in degrees])


            remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
            G.remove_nodes_from(remove)

            N_nodes_new = list(G.nodes)    #list of nodes with k>0


            H = G.copy()   # graph contains nodes that have k > 0
            I = G.copy()
            degree_sequence_erdos = list(H.degree())


            selected_nodes = []
            for node in H.nodes:
                neighbor_list = [n for n in H.neighbors(node)]
                selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes

            avg_degree_selected_nodes = np.mean([H.degree(i) for i in selected_nodes])

            color_dict, avg_color_degrees_dict = avg_color_degree(H)
            # print(min(avg_color_degrees_dict))

#             immune_nodes = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
            immune_nodes = get_immune_nodes(H, selected_nodes)
            # print(len(immune_nodes))
#             print(" theta =", thetha," ", "# selected nodes =", len(selected_nodes), " ", "immune nodes =", len(immune_nodes), " ", "avg. degree of selected nodes =", avg_degree_selected_nodes)

            failed_nodes_at_the_end_cascade = cascading_failure(G,H,immune_nodes,thetha,protection_prob)

            suriviving_prob = (len(I.nodes) - len(failed_nodes_at_the_end_cascade))/ len(I.nodes)
            intermediate_suriviving_prob_array.append(suriviving_prob)
            if protection_prob ==1.0:
                intermediate_immune_node_size.append(len(immune_nodes)/N_nodes)
                intermediate_selected_node_size.append(len(selected_nodes)/N_nodes)
        suriviving_prob_array.append(round(np.mean(intermediate_suriviving_prob_array),2))
        if protection_prob ==1.0:
            immune_node_size.append(np.mean(intermediate_immune_node_size))
            selected_node_size.append(np.mean(intermediate_selected_node_size))
    print(suriviving_prob_array)
    plt.plot(Phi, suriviving_prob_array, label = protection_prob)

plt.plot(Phi,immune_node_size,'--', label = "immune" )
# plt.plot(Phi,selected_node_size,':', label = "Selected" )
print(immune_node_size)
print(Phi)
plt.xlabel("Threshold")
plt.ylabel("System's surviving probability")
plt.legend(loc ="upper right")
plt.title("ER Network  "+"N = "+str(N_nodes)+" , " "<k> = "+str(average_degree_of_original_graph))
plt.show()


In [ ]:
# Fig 8 (b): This will give you the Smolyak's strategy surviving prob. for fully protection curve (SF) 
# for comparion, you may use Fig_5 code


###################### ER coloring algorith #########################
def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -1) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 1) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
    
    return None

def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(H.degree())      # in original code, it's "G.degree()"
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict


import networkx as nx

def get_immune_nodes(G, fragile_nodes):
    """
    Generate the connected subgraph of fragile nodes, calculate the average degree 
    of the subgraph, and return the fragile nodes whose degree is greater than the 
    average degree of that subgraph.

    Arguments:
    - G (networkx.Graph): The original graph.
    - fragile_nodes (list): List of fragile nodes in the network.

    Returns:
    - immune_nodes (set): Set of fragile nodes whose degree is greater than the 
                           average degree of the connected subgraph of fragile nodes.
    """
    
    # Create a subgraph of only the fragile nodes
    subgraph = G.subgraph(fragile_nodes).copy()
    
    # Calculate the average degree of the subgraph
    avg_degree = sum(dict(subgraph.degree()).values()) / len(subgraph.nodes())
    
    # Identify immune nodes (nodes with degree greater than the average degree)
    immune_nodes = {node for node in subgraph.nodes() if subgraph.degree(node) > avg_degree}
    
    return immune_nodes





def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    # print(min_value)
    protected_nodes_list = []
    for i in failed_nodes:
        if H.degree(i) >min_value:
            protected_nodes_list.append(i)
    # print(protected_nodes_list)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha, prot_prob):
################################ Cascading failure ######################################
    #print(H.degree())
    #initial_edge = random.choice(list(H.edges))
   
    initial_node = random.choice([i for i in list(H.nodes) if i not in immune_nodes ])
   
    #print("edge removed = ", initial_edge)
    #H.remove_edge(*initial_edge)
    H.remove_node(initial_node)         #randomly removing a node, avoiding the immume_nodes, to initiate the cascade
#     print("node removed =",initial_node)
    #print(H.degree())

    
    not_failed_nodes =[]
    failed_nodes = []
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        #print("current_failed_nodes = ", current_failed_nodes)
        #print("current_failed_nodes degree =", H.degree())
        for each in current_failed_nodes:
            #print("entered for loop: current_failed_nodes")
            if each in immune_nodes:
                if prot_prob ==1.0:
                    pass
                elif prot_prob ==0.0:
                    H.remove_node(each)
                    if each not in failed_nodes:
                        failed_nodes.append(each)
                else:
                    r = np.random.random(10000)
                    event = (r<prot_prob)
                    ratio = np.mean(event)
                    if np.random.choice(event) == False:
                        H.remove_node(each)
                        if each not in failed_nodes:
                            failed_nodes.append(each)
            else :
                #print("entered else remove loop: node removed = ", each)
                #print(each, "Non -immune node removed")
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
#                 current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]

        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}
        #print("not_failed_nodes = ",not_failed_nodes)
        #print("not_failed_nodes_desc = ",not_failed_nodes_desc)
        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or (each in immune_nodes and prot_prob==1.0) or (each in immune_nodes and prot_prob==0.7) or (each in immune_nodes and prot_prob==0.4):
                #print("entered if condition: each>= phi or each in immune")
                count +=1
        if count == len(not_failed_nodes):
            #print("Break: count == len(not_failed_nodes)")
            break
         
    return failed_nodes


N_nodes = 20000
# p = 0.008
m_parameter = 3
average_degree_of_original_graph = 2*m_parameter
prot_prob = [1.0]

Phi = np.arange(0.82,0.90,0.005)


realizations = 150
immune_node_size =[] 
selected_node_size =[]

for protection_prob in prot_prob:
    suriviving_prob_array = []
    for thetha in Phi:
        intermediate_suriviving_prob_array = []
        intermediate_immune_node_size =[]
        intermediate_selected_node_size =[]
        for i in range(realizations):
            G = nx.barabasi_albert_graph(N_nodes,m_parameter)
            degree_sequence_erdos = list(G.degree())
            degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
            avg_degree = np.mean([int(x) for x in degrees])


            remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
            G.remove_nodes_from(remove)

            N_nodes_new = list(G.nodes)    #list of nodes with k>0


            H = G.copy()   # graph contains nodes that have k > 0
            I = G.copy()
            degree_sequence_erdos = list(H.degree())


            selected_nodes = []
            for node in H.nodes:
                neighbor_list = [n for n in H.neighbors(node)]
                selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes

            avg_degree_selected_nodes = np.mean([H.degree(i) for i in selected_nodes])

            color_dict, avg_color_degrees_dict = avg_color_degree(H)
            # print(min(avg_color_degrees_dict))

#             immune_nodes = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
            immune_nodes = get_immune_nodes(H, selected_nodes)
            # print(len(immune_nodes))
#             print(" theta =", thetha," ", "# selected nodes =", len(selected_nodes), " ", "immune nodes =", len(immune_nodes), " ", "avg. degree of selected nodes =", avg_degree_selected_nodes)

            failed_nodes_at_the_end_cascade = cascading_failure(G,H,immune_nodes,thetha,protection_prob)

            suriviving_prob = (len(I.nodes) - len(failed_nodes_at_the_end_cascade))/ len(I.nodes)
            intermediate_suriviving_prob_array.append(suriviving_prob)
            if protection_prob ==1.0:
                intermediate_immune_node_size.append(len(immune_nodes)/N_nodes)
                intermediate_selected_node_size.append(len(selected_nodes)/N_nodes)
        suriviving_prob_array.append(round(np.mean(intermediate_suriviving_prob_array),2))
        if protection_prob ==1.0:
            immune_node_size.append(np.mean(intermediate_immune_node_size))
            selected_node_size.append(np.mean(intermediate_selected_node_size))
    print(suriviving_prob_array)
    plt.plot(Phi, suriviving_prob_array, label = protection_prob)

plt.plot(Phi,immune_node_size,'--', label = "immune" )
# plt.plot(Phi,selected_node_size,':', label = "Selected" )
print(immune_node_size)
print(Phi)
plt.xlabel("Threshold")
plt.ylabel("System's surviving probability")
plt.legend(loc ="upper right")
plt.title("SF Network  "+"N = "+str(N_nodes)+" , " "<k> = "+str(average_degree_of_original_graph))
plt.show()
